## Evaluation of the Solution Space under different conditions

In [ ]:
# Ensure necessary packages are installed
!pip install cobra efmtool numpy pandas scipy

Definition of the EV Network from Eberhard Voit's "A first course in Systems Biology"

In [2]:
from cobra import Model, Reaction, Metabolite

# Modell erstellen
model = Model("Toy_Network")

# Metaboliten definieren (intern)
A = Metabolite("A", compartment="c")
B = Metabolite("B", compartment="c")
C = Metabolite("C", compartment="c")
D = Metabolite("D", compartment="c")
E = Metabolite("E", compartment="c")

# Metaboliten definieren (extern)
A_e = Metabolite("A_e", compartment="e")
B_e = Metabolite("B_e", compartment="e")
D_e = Metabolite("D_e", compartment="e")
E_e = Metabolite("E_e", compartment="e")

# Reaktionen definieren
r1_in = Reaction("r1_in")
r1_in.name = "A import"
r1_in.lower_bound = 0
r1_in.upper_bound = 1000
r1_in.add_metabolites({A_e: -1, A: 1})

r2 = Reaction("r2")
r2.name = "A to C"
r2.lower_bound = 0
r2.upper_bound = 1000
r2.add_metabolites({A: -1, C: 1})

r3 = Reaction("r3")
r3.name = "C to D and E"
r3.lower_bound = 0
r3.upper_bound = 1000
r3.add_metabolites({C: -1, D: 1, E: 1})

r4_out = Reaction("r4_out")
r4_out.name = "D export"
r4_out.lower_bound = 0
r4_out.upper_bound = 1000
r4_out.add_metabolites({D: -1, D_e: 1})

r5 = Reaction("r5")
r5.name = "A to B"
r5.lower_bound = 0
r5.upper_bound = 1000
r5.add_metabolites({A: -1, B: 1})

r6r = Reaction("r6r")
r6r.name = "B to C"
r6r.lower_bound = -1000
r6r.upper_bound = 1000
r6r.add_metabolites({B: -1, C: 1})

r7 = Reaction("r7")
r7.name = "B to D"
r7.lower_bound = 0
r7.upper_bound = 1000
r7.add_metabolites({B: -1, D: 1})

r8r_out = Reaction("r8r_out")
r8r_out.name = "B import"
r8r_out.lower_bound = -1000
r8r_out.upper_bound = 1000
r8r_out.add_metabolites({B_e: -1, B: 1})

r9_out = Reaction("r9_out")
r9_out.name = "E export"
r9_out.lower_bound = 0
r9_out.upper_bound = 1000
r9_out.add_metabolites({E: -1, E_e: 1})

# Reaktionen zum Modell hinzufügen
model.add_reactions([r1_in, r2, r3, r4_out, r5, r6r, r7, r8r_out, r9_out])

# Modell überprüfen
print("Reaktionen:", [rxn.id for rxn in model.reactions])
print("Metaboliten:", [met.id for met in model.metabolites])
print("Gene:", [gene.id for gene in model.genes])

# Modell speichern (optional)
#model.save("toy_network.xml")


Reaktionen: ['r1_in', 'r2', 'r3', 'r4_out', 'r5', 'r6r', 'r7', 'r8r_out', 'r9_out']
Metaboliten: ['A_e', 'A', 'C', 'D', 'E', 'D_e', 'B', 'B_e', 'E_e']
Gene: []


In [3]:
import numpy as np
import cobra


def get_stoichiometric_matrix(model):
    # Filtere die Reaktionen, um nur Reaktionen zu behalten, die nicht mit "EX_" beginnen
    internal_reactions = [rxn for rxn in model.reactions if not rxn.id.startswith("EX_")]

    # Extrahiere die Metaboliten aus dem Modell
    internal_metabolites = [meta for meta in model.metabolites if not meta.compartment == 'e']

    # Erstelle die leere Stöchiometrie-Matrix mit (Anzahl der Metaboliten, Anzahl der internen Reaktionen)
    stoichiometric_matrix = np.zeros((len(internal_metabolites), len(internal_reactions)))

    # Befülle die Stöchiometrie-Matrix so, dass jede Spalte eine Reaktion repräsentiert
    for j, rxn in enumerate(internal_reactions):  # Spaltenweise über Reaktionen iterieren
        for i, met in enumerate(internal_metabolites):  # Zeilenweise über Metaboliten iterieren
            stoichiometric_matrix[i, j] = rxn.metabolites.get(met, 0)  # Falls Metabolit nicht in der Reaktion ist, wird 0 zurückgegeben
    return stoichiometric_matrix


In [4]:
import efmtool
import cobra
import numpy as np
from efmtool import calculate_efms

def cobra_to_efms(model):
    # 1. Stoichiometric matrix (S matrix)
    metabolites = [
    metabolite for metabolite in model.metabolites 
    if metabolite.compartment == "c"
    ]
    reactions = model.reactions
    stoichiometry = np.zeros((len(metabolites), len(reactions)))
    #stoichiometry = get_stoichiometric_matrix(model)
    
    for i, met in enumerate(metabolites):
        for j, rxn in enumerate(reactions):
            try:
                stoichiometry[i, j] = rxn.get_coefficient(met.id)
            except KeyError:
                
                stoichiometry[i, j] = 0.0
    
    print("Stoichiometric matrix shape:", stoichiometry.shape)
    print("Stoichiometric matrix:", stoichiometry)

    # 2. Reversibilities: 1 for reversible, 0 for irreversible reactions
    reversibilities = [1 if rxn.reversibility else 0 for rxn in reactions]
    
    print("Reversibilities:", reversibilities)
    print("Number of reactions:", len(reactions))

    # 3. Reaction names
    reaction_names = [rxn.id for rxn in reactions]
    
    # 4. Metabolite names
    metabolite_names = [met.id for met in metabolites]
    
    return calculate_efms(stoichiometry, reversibilities, reaction_names, metabolite_names), reaction_names





# 6. Berechne die Elementarmoden (EFMs) mit dem efmtool und Reaktionsnamen
efms, reaction_names = cobra_to_efms(model)

# 7. Ergebnisse anzeigen
print("Berechnete Elementarmoden:")
print(np.column_stack((reaction_names, efms)))


Stoichiometric matrix shape: (5, 9)
Stoichiometric matrix: [[ 1. -1.  0.  0. -1.  0.  0.  0.  0.]
 [ 0.  1. -1.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  1. -1.  0.  0.  1.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0. -1.]
 [ 0.  0.  0.  0.  1. -1. -1.  1.  0.]]
Reversibilities: [0, 0, 0, 0, 0, 1, 0, 1, 0]
Number of reactions: 9
2025-01-03  21:33:10.433  main                     INFO     | logger initialized
2025-01-03  21:33:10.434  main                     INFO     | =====================================================
2025-01-03  21:33:10.434  main                     INFO     | efmtool version 4.7.1, 2009-12-04 18:30:05
2025-01-03  21:33:10.435  main                     INFO     | Copyright (c) 2009, Marco Terzer, Zurich, Switzerland
2025-01-03  21:33:10.435  main                     INFO     | This is free software, !!! NO WARRANTY !!!
2025-01-03  21:33:10.435  main                     INFO     | See LICENCE.txt for redistribution conditions
2025-01-03  21:33:10.435  main                

In [5]:
import numpy as np


input_array = np.column_stack((reaction_names, efms))

# Extract headers and values
headers = input_array[:, 0]
values = input_array[:, 1:].astype(float)  # Convert values to float

# Construct dictionaries
experimental_data = []
for col_idx in range(values.shape[1]):
    data_point = {}
    for row_idx, header in enumerate(headers):
        if header.endswith("_in") or header.endswith("_out"):
            # Add only reactions ending with "_in" or "_out"
            data_point[header] = values[row_idx, col_idx]
    # Normalize so "A_in" is always 1.0
    if "A_in" in data_point and data_point["A_in"] != 0.0:
        normalization_factor = data_point["A_in"]
        for key in data_point.keys():
            data_point[key] /= normalization_factor
    experimental_data.append(data_point)

# Output the result
for data in experimental_data:
    print(data)

experimental_data_zeropadded_out = experimental_data

{'r1_in': 1.0, 'r4_out': 0.0, 'r8r_out': -1.0, 'r9_out': 0.0}
{'r1_in': 1.0, 'r4_out': 0.0, 'r8r_out': -1.0, 'r9_out': 0.0}
{'r1_in': 1.0, 'r4_out': 1.0, 'r8r_out': 0.0, 'r9_out': 0.0}
{'r1_in': 0.0, 'r4_out': 1.0, 'r8r_out': 1.0, 'r9_out': 0.0}
{'r1_in': 1.0, 'r4_out': 1.0, 'r8r_out': 0.0, 'r9_out': 1.0}
{'r1_in': 1.0, 'r4_out': 1.0, 'r8r_out': 0.0, 'r9_out': 1.0}
{'r1_in': 1.0, 'r4_out': 1.0, 'r8r_out': 0.0, 'r9_out': 0.0}
{'r1_in': 0.0, 'r4_out': 1.0, 'r8r_out': 1.0, 'r9_out': 1.0}


In [6]:
import numpy as np
from scipy.optimize import lsq_linear
from scipy.optimize import minimize





#print(reaction_ids)
# Funktion zur Ableitung des Zielvektors aus den experimentellen Daten
def create_target_vector(data_point, reaction_ids):
    target_flux = np.full(len(reaction_ids), np.nan)  # Vektor mit NaN initialisieren
    for rxn_id, value in data_point.items():
        if rxn_id in reaction_ids:  # Nur Reaktionen, die im Modell existieren
            idx = reaction_ids.index(rxn_id)
            target_flux[idx] = value  # Setze den Zielwert
    return target_flux


def least_square_min(model, S, experimental_data):
    reaction_ids = [rxn.id for rxn in model.reactions if not rxn.id.startswith("EX_")]
    total_score = 0.0
    successful_optimizations = 0

    for data_point in experimental_data:
        # Zielvektor für aktuellen Datenpunkt erstellen
        target_flux = create_target_vector(data_point, reaction_ids)

        # Filter für bekannte Flüsse (Reaktionen, die in 'target_flux' nicht NaN sind)
        known_flux_indices = ~np.isnan(target_flux)
        target_flux_values = target_flux[known_flux_indices]

        def objective(x):
            return np.linalg.norm(x[known_flux_indices] - target_flux_values)

        constraints = {'type': 'eq', 'fun': lambda x: np.dot(S, x)}

        initial_guess = np.zeros(len(reaction_ids))
        bounds = [(model.reactions.get_by_id(rxn_id).lower_bound,
                   model.reactions.get_by_id(rxn_id).upper_bound)
                  for rxn_id in reaction_ids]

        result = minimize(objective, initial_guess, constraints=constraints, bounds=bounds, tol=1e-10)

        if result.success:
            optimized_flux = result.x
            total_score += np.linalg.norm(optimized_flux[known_flux_indices] - target_flux_values)
            successful_optimizations += 1
        else:
            print(f"Optimization failed for data point: {data_point}. Message: {result.message}")

    # Return None if any optimization failed, otherwise the total score
    if successful_optimizations < len(experimental_data):
        return 0.0, len(experimental_data)
    else:
        return total_score, len(experimental_data)

 
S = get_stoichiometric_matrix(model)
print(least_square_min(model, S, experimental_data_zeropadded_out))

(3.123498217099662e-08, 8)


In [7]:
import random
from cobra import Reaction

# Erstellen von Reaktionen mit 1-2 Substrat(en) und 1-2 Produkt(en)
def generate_random_reactions(model, num_reactions=2000):
    # Filter out metabolites that end with '_EX'
    metabolites = [met for met in model.metabolites if not met.compartment == 'e']
    random_reactions = []

    for i in range(num_reactions):
        reaction_valid = False

        while not reaction_valid:
            # Zufällige Auswahl der Anzahl an Substraten und Produkten
            num_reactants = random.randint(1, 2)
            num_products = random.randint(1, 2)

            # Zufällige Auswahl der Substrate und Produkte
            reactants = random.sample(metabolites, num_reactants)
            products = random.sample(metabolites, num_products)

            # Check, dass auf beiden Seiten Metaboliten vorhanden sind
            if set(reactants).isdisjoint(set(products)):
                reaction_valid = True

                # Erstellung der Reaktion
                reaction = Reaction(f'Random_Rxn_{i+1}')
                reaction.name = f'Random Reaction {i+1}'

                reaction.add_metabolites({
                    met: -1.0 for met in reactants
                })
                reaction.add_metabolites({
                    met: 1.0 for met in products
                })

                # 50% Chance, die Reaktion reversibel zu machen
                if random.random() < 0.5:
                    reaction.lower_bound = -1000.0
                else:
                    reaction.lower_bound = 0.0

                reaction.upper_bound = 1000.0

                random_reactions.append(reaction)

    return random_reactions

# Generierung von 2000 zufälligen Reaktionen + den Inversen
random_reactions_list = generate_random_reactions(model, num_reactions=2000)

# Anzahl der insgesamt generierten Reaktionen
print(f"\nTotal number of random reactions generated: {len(random_reactions_list)}")


Total number of random reactions generated: 2000


In [8]:
import cobra
import random

def sample_reactions(model, n_reactions):
  
    # Filter reactions that do not end with '_in' or '_out'
    valid_reactions = [rxn for rxn in model.reactions if not rxn.id.endswith('_in') and not rxn.id.endswith('_out')]

    # Ensure that the model has enough valid reactions
    if n_reactions > len(valid_reactions):
        raise ValueError("Requested number of reactions exceeds the total number of valid reactions in the model")

    # Sample reactions randomly from the valid reactions list
    sampled_reactions = random.sample(valid_reactions, n_reactions)

    for rxn in sampled_reactions:
        print(rxn.id)
    
    return sampled_reactions


In [9]:
import random

def sample_reactions_from_list(reactions_list, n_reactions):
    
    # Ensure the list has enough reactions to sample from
    if n_reactions > len(reactions_list):
        raise ValueError("Requested number of reactions exceeds the total number of reactions in the list")

    # Sample reactions randomly from the given list
    sampled_reactions = random.sample(reactions_list, n_reactions)

    for rxn in sampled_reactions:
        print(rxn.id if hasattr(rxn, 'id') else rxn)  # Print the ID if the object has one

    return sampled_reactions


In [10]:
def model_edit_distance(model1, model2):
    
    # Get sets of reaction IDs for both models
    reactions_model1 = {rxn.id for rxn in model1.reactions}
    reactions_model2 = {rxn.id for rxn in model2.reactions}
    
    # Calculate the symmetric difference (reactions present in one model but not the other)
    differing_reactions = reactions_model1.symmetric_difference(reactions_model2)
    
    # The edit distance is the number of differing reactions
    edit_distance = len(differing_reactions)
    
    return edit_distance




In [11]:
import pandas as pd

def filter_and_display(list1, list2, list3):
    
    # Erstellen des DataFrames
    df = pd.DataFrame({'Scores': list1, 'Edit Distances': list2, 'Computation Count': list3})
    
    # Filtern: Entfernen von Zeilen mit NaN oder 0.0 in der ersten Liste
    filtered_df = df.dropna(subset=['Scores'])
    filtered_df = filtered_df[filtered_df['Scores'] != 0.0]
    
    # Ergebnis zurückgeben oder anzeigen
    return filtered_df




## Generation of data points using sampling

In [12]:
import cobra
from cobra.sampling import sample

import pandas as pd


# Function to normalize a row based on A_in, excluding boundary reactions
def normalize_row(row, normalization_id):
    normalization_factor = row[normalization_id]
    # Exclude boundary reactions (columns starting with "EX_" or ending with "_out")
    filtered_row = {col: value for col, value in row.items() if (col.endswith("_in") or col.endswith("_out"))}
    return {col: (value / normalization_factor) if normalization_factor != 0 else 0.0
            for col, value in filtered_row.items()}


def create_synthetic_datapoints(model, amount, normalization_id):
    s = sample(model, amount)
    
    normalized_data = [normalize_row(row, normalization_id) for _, row in s.iterrows()]
    
    return normalized_data

In [13]:
model_data_gen = model.copy()

# Exchange Reaktionen für extracelluläre Metaboliten
model_data_gen.add_boundary(A_e, type="exchange")
model_data_gen.add_boundary(B_e, type="exchange")
model_data_gen.add_boundary(D_e, type="exchange")
model_data_gen.add_boundary(E_e, type="exchange")

Reaction identifier,EX_E_e
Name,exchange
Memory address,0x284d495d0
Stoichiometry,E_e <=> <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [14]:
import cobra

def check_for_deadends(model):    

    
    # Lists for results
    upstream_no_consumption_roots = []
    downstream_no_production_roots = []

    # Initialize dictionaries for production and consumption of metabolites
    metabolite_produced = {met: False for met in model.metabolites if met.compartment == 'c'}
    metabolite_consumed = {met: False for met in model.metabolites if met.compartment == 'c'}

    # Iterate over all reactions in the model
    for reaction in model.reactions:
        if reaction.reversibility:
            #print(reaction)
            # If reversible, treat all metabolites as both reactants and products
            for met in reaction.metabolites:
                metabolite_produced[met] = True
                metabolite_consumed[met] = True
        else:
            # Metabolites produced in this reaction
            for product in reaction.products:
                #print(reaction.products)
                if product.compartment == 'c':  # Only consider intracellular metabolites
                    metabolite_produced[product] = True

            # Metabolites consumed in this reaction
            for reactant in reaction.reactants:
                
                if reactant.compartment == 'c':  # Only consider intracellular metabolites
                    metabolite_consumed[reactant] = True

    # Determine Upstream No-Consumption Roots (produced but not consumed)
    for met, produced in metabolite_produced.items():
        if produced and not metabolite_consumed[met]:
            upstream_no_consumption_roots.append(met.id)

    # Determine Downstream No-Production Roots (consumed but not produced)
    for met, consumed in metabolite_consumed.items():
        if consumed and not metabolite_produced[met]:
            downstream_no_production_roots.append(met.id)

    #print("Upstream No-Consumption Roots:", upstream_no_consumption_roots)
    #print("Downstream No-Production Roots:", downstream_no_production_roots)

    return len(upstream_no_consumption_roots) + len(downstream_no_production_roots)




In [15]:
def filter_datapoints(datapoints, reaction_list):
    
    
    ids = [reaction.id for reaction in reaction_list]
    
    filtered_data = []

    for datapoint in datapoints:
        filtered_datapoint = {key: value for key, value in datapoint.items() if key in ids}
        filtered_data.append(filtered_datapoint)

    return filtered_data

In [16]:
from itertools import combinations

def evaluate_reaction_subsets(model, experimental_data, random_reactions, num_removed):
    
    
    threshold = len(experimental_data)/10 * 3e-7
    
    # Step 1: Remove 2 reactions from the model
    model_copy = model.copy()
    valid_reactions = [rxn for rxn in model_copy.reactions if not rxn.id.endswith('_in') and not rxn.id.endswith('_out')]
    removed_reactions = random.sample(valid_reactions, num_removed)
    model_copy.remove_reactions(removed_reactions)
    print(model_copy.reactions)

    experimental_data = filter_datapoints(experimental_data, model_copy.reactions)
    
    # Step 2: Assemble reaction pool of reactions that can be added
    reaction_pool = removed_reactions + random_reactions

    # Step 3: Evaluate subsets of size 2 and 3
    adequate_subsets_count = 0
    num_subsets_unblocked = 0
    for subset_size in [2, 3]:
        for subset in combinations(reaction_pool, subset_size):
            # Add the subset of reactions to the model
            temp_model = model_copy.copy()
            temp_model.add_reactions(subset)

            # Evaluate the score
            S = get_stoichiometric_matrix(temp_model)
            score, _ = least_square_min(temp_model, S, experimental_data)
            
            # Check if the score meets the criteria
            if score < threshold and score != 0.0:
                adequate_subsets_count += 1
            
                num_deadends = check_for_deadends(temp_model)
                if num_deadends == 0:
                    num_subsets_unblocked += 1
                
    return adequate_subsets_count, num_subsets_unblocked




In [18]:
random.seed(42)
np.random.seed(42)

In [19]:
test_random_reacs = generate_random_reactions(model, 13)
#remove_reacs = sample_reactions(model, 2)

In [20]:
random.seed(42)
np.random.seed(42)

In [21]:
synth_data20 = create_synthetic_datapoints(model_data_gen, 20, "r1_in")

In [22]:
random.seed(42)
np.random.seed(42)

In [23]:
synth_data100 = create_synthetic_datapoints(model_data_gen, 100, "r1_in")

In [24]:
random.seed(42)
np.random.seed(42)

In [25]:
synth_data500 = create_synthetic_datapoints(model_data_gen, 500, "r1_in")

In [26]:
random.seed(17)
np.random.seed(17)

## Evaluating number of adequate subsets for different experimental data

In [27]:
num_adequate_subsets_efm, num_unblocked_efm = evaluate_reaction_subsets(model, experimental_data_zeropadded_out, test_random_reacs, 2)

[<Reaction r1_in at 0x283195990>, <Reaction r2 at 0x284d4a990>, <Reaction r3 at 0x28543c810>, <Reaction r4_out at 0x28543c910>, <Reaction r5 at 0x28543e050>, <Reaction r8r_out at 0x28543e2d0>, <Reaction r9_out at 0x28543e490>]


In [28]:
print(f"Number of subsets with adequate scores: {num_adequate_subsets_efm}")
print(f"Number of unblocked models: {num_unblocked_efm}")

Number of subsets with adequate scores: 73
Number of unblocked models: 73


In [29]:
random.seed(17)
np.random.seed(17)

In [30]:
num_adequate_subsets20, num_unblocked_20 = evaluate_reaction_subsets(model, synth_data20, test_random_reacs, 2)
print(f"Number of subsets with adequate scores: {num_adequate_subsets20}")
print(f"Number of unblocked models: {num_unblocked_20}")

[<Reaction r1_in at 0x2866fbd10>, <Reaction r2 at 0x286700090>, <Reaction r3 at 0x286700210>, <Reaction r4_out at 0x286700390>, <Reaction r5 at 0x286700510>, <Reaction r8r_out at 0x2867009d0>, <Reaction r9_out at 0x2866fb890>]
Number of subsets with adequate scores: 78
Number of unblocked models: 78


In [31]:
random.seed(17)
np.random.seed(17)

In [32]:
num_adequate_subsets100, num_unblocked_100 = evaluate_reaction_subsets(model, synth_data100, test_random_reacs, 2)
print(f"Number of subsets with adequate scores: {num_adequate_subsets100}")
print(f"Number of unblocked models: {num_unblocked_100}")

[<Reaction r1_in at 0x2871ce510>, <Reaction r2 at 0x2871af3d0>, <Reaction r3 at 0x28598f090>, <Reaction r4_out at 0x2871cdd10>, <Reaction r5 at 0x287267cd0>, <Reaction r8r_out at 0x287265010>, <Reaction r9_out at 0x287266490>]
Number of subsets with adequate scores: 74
Number of unblocked models: 74


In [33]:
random.seed(17)
np.random.seed(17)

In [34]:
num_adequate_subsets500, num_unblocked_500 = evaluate_reaction_subsets(model, synth_data500, test_random_reacs, 2)
print(f"Number of subsets with adequate scores: {num_adequate_subsets500}")
print(f"Number of unblocked models: {num_unblocked_500}")

[<Reaction r1_in at 0x28574fe10>, <Reaction r2 at 0x28574cf90>, <Reaction r3 at 0x28574ce90>, <Reaction r4_out at 0x28574d0d0>, <Reaction r5 at 0x28574cc90>, <Reaction r8r_out at 0x28574cb90>, <Reaction r9_out at 0x28574c8d0>]
Number of subsets with adequate scores: 74
Number of unblocked models: 74


Different Seed

In [35]:
random.seed(23)
np.random.seed(23)

In [36]:
num_adequate_subsets_efm23, num_unblocked_efm_23 = evaluate_reaction_subsets(model, experimental_data_zeropadded_out, test_random_reacs, 2)
print(f"Number of subsets with adequate scores: {num_adequate_subsets_efm23}")
print(f"Number of unblocked models: {num_unblocked_efm_23}")

[<Reaction r1_in at 0x28619e9d0>, <Reaction r3 at 0x28567f090>, <Reaction r4_out at 0x28619e450>, <Reaction r6r at 0x28619c990>, <Reaction r7 at 0x28619c8d0>, <Reaction r8r_out at 0x28619cd10>, <Reaction r9_out at 0x28619c5d0>]
Number of subsets with adequate scores: 335
Number of unblocked models: 335


In [37]:
random.seed(23)
np.random.seed(23)

In [38]:
num_adequate_subsets20_23, num_unblocked_20_23 = evaluate_reaction_subsets(model, synth_data20, test_random_reacs, 2)
print(f"Number of subsets with adequate scores: {num_adequate_subsets20_23}")
print(f"Number of unblocked models: {num_unblocked_20_23}")

[<Reaction r1_in at 0x285eca210>, <Reaction r3 at 0x285eca150>, <Reaction r4_out at 0x285eca250>, <Reaction r6r at 0x285ecaf10>, <Reaction r7 at 0x285ecaf90>, <Reaction r8r_out at 0x285ec85d0>, <Reaction r9_out at 0x285ec8b90>]
Number of subsets with adequate scores: 333
Number of unblocked models: 333


In [39]:
random.seed(23)
np.random.seed(23)

In [40]:
num_adequate_subsets100_23, num_unblocked_100_23 = evaluate_reaction_subsets(model, synth_data100, test_random_reacs, 2)
print(f"Number of subsets with adequate scores: {num_adequate_subsets100_23}")
print(f"Number of unblocked models: {num_unblocked_100_23}")

[<Reaction r1_in at 0x285f60c50>, <Reaction r3 at 0x285f63cd0>, <Reaction r4_out at 0x285f60e90>, <Reaction r6r at 0x285f61650>, <Reaction r7 at 0x285f602d0>, <Reaction r8r_out at 0x285f60150>, <Reaction r9_out at 0x285f63f10>]
Number of subsets with adequate scores: 346
Number of unblocked models: 346


In [41]:
random.seed(23)
np.random.seed(23)

In [42]:
num_adequate_subsets500_23, num_unblocked_500_23 = evaluate_reaction_subsets(model, synth_data500, test_random_reacs, 2)
print(f"Number of subsets with adequate scores: {num_adequate_subsets500_23}")
print(f"Number of unblocked models: {num_unblocked_500_23}")

[<Reaction r1_in at 0x285f60cd0>, <Reaction r3 at 0x285f63e10>, <Reaction r4_out at 0x285dbac50>, <Reaction r6r at 0x285db8b10>, <Reaction r7 at 0x285db95d0>, <Reaction r8r_out at 0x285db9ed0>, <Reaction r9_out at 0x286418c90>]
Number of subsets with adequate scores: 348
Number of unblocked models: 348


## Evaluation model reconstruction taking into account internal flux values

In [43]:
import cobra
from cobra.sampling import sample

import pandas as pd


# Function to normalize a row based on a normalization id, excluding boundary reactions
def normalize_row_internal(row, normalization_id):
    normalization_factor = row[normalization_id]
    # Exclude boundary reactions (columns starting with "EX_" or ending with "_out")
    filtered_row = {col: value for col, value in row.items() if not (col.startswith("EX_"))}
    return {col: (value / normalization_factor) if normalization_factor != 0 else 0.0
            for col, value in filtered_row.items()}


def create_synthetic_datapoints_internal(model, amount, normalization_id):
    s = sample(model, amount)
    
    normalized_data = [normalize_row_internal(row, normalization_id) for _, row in s.iterrows()]
    
    return normalized_data

In [44]:
random.seed(42)
np.random.seed(42)

In [45]:
synth_data_internal42 = create_synthetic_datapoints_internal(model_data_gen, 20, "r1_in")

print(synth_data_internal42)

[{'r1_in': 1.0, 'r2': 0.251411895982439, 'r3': 0.3089745105705202, 'r4_out': 0.47836539858353555, 'r5': 0.7485881040175594, 'r6r': 0.05756261458808033, 'r7': 0.16939088801301433, 'r8r_out': -0.5216346014164654, 'r9_out': 0.3089745105705202}, {'r1_in': 1.0, 'r2': 0.07260286846284122, 'r3': 0.721849922334519, 'r4_out': 0.7290900475099578, 'r5': 0.9273971315371572, 'r6r': 0.6492470538716782, 'r7': 0.007240125175437548, 'r8r_out': -0.27090995249004024, 'r9_out': 0.721849922334519}, {'r1_in': 1.0, 'r2': 0.6523473466570071, 'r3': 0.05161825411092367, 'r4_out': 0.9483571989511526, 'r5': 0.34765265334299067, 'r6r': -0.6007290925460829, 'r7': 0.8967389448402269, 'r8r_out': -0.05164280104884401, 'r9_out': 0.05161825411092367}, {'r1_in': 1.0, 'r2': 0.9207240642659905, 'r3': 0.4901316463442748, 'r4_out': 1.1007338776593896, 'r5': 0.07927593573400625, 'r6r': -0.43059241792171354, 'r7': 0.6106022313151098, 'r8r_out': 0.10073387765938996, 'r9_out': 0.4901316463442748}, {'r1_in': 1.0, 'r2': 0.42982373

In [46]:
random.seed(17)
np.random.seed(17)

In [47]:
num_adequate_subsets_internal17 = evaluate_reaction_subsets(model, synth_data_internal42, test_random_reacs, 2)
print(f"Number of subsets with adequate scores: {num_adequate_subsets_internal17}")

[<Reaction r1_in at 0x285c28650>, <Reaction r2 at 0x285c28690>, <Reaction r3 at 0x285c28450>, <Reaction r4_out at 0x285c2b4d0>, <Reaction r5 at 0x285c2b610>, <Reaction r8r_out at 0x285c2b150>, <Reaction r9_out at 0x285c29410>]
Number of subsets with adequate scores: (41, 41)


In [50]:
random.seed(23)
np.random.seed(23)

In [51]:
num_adequate_subsets_internal23 = evaluate_reaction_subsets(model, synth_data_internal42, test_random_reacs, 2)
print(f"Number of subsets with adequate scores: {num_adequate_subsets_internal23}")

[<Reaction r1_in at 0x2855eab50>, <Reaction r3 at 0x2854fcd50>, <Reaction r4_out at 0x2854fdb50>, <Reaction r6r at 0x2854fcb50>, <Reaction r7 at 0x2854fe010>, <Reaction r8r_out at 0x2854fcad0>, <Reaction r9_out at 0x2854fff90>]
Number of subsets with adequate scores: (56, 56)
